In [1]:
# Задача определить абонентов (msisdn), которые являются одной и той же Персоной (человеком).
import csv
import pandas as pd
import numpy as np
pd.options.display.max_columns = 1000

In [7]:
# Информация о перемещении случайного набора абонентов за период 23-25 мая 2013 года
# в котором точно присутствуют номера телефонов, принадлежащие одному и тому же человеку.
ints_a=["lac","cid","msisdn","event_type","tstamp","tac1","tac2","tac3","tac4","tac5","tac6","tac7","tac8"]
floats_a=["imei","max_dist","long","lat","start_angle","end_angle"]
texts_a=["cell_type"]
df_a = pd.read_csv('../input/02_Data_test.csv', sep=";")
for i in range(1,9):
    df_a["tac"+str(i)] = df_a["imei"].apply(lambda s: str(round(s,0))[:i] if len(str(s))>=i and s == s else "-1")
for col in ints_a:
    df_a[col] = df_a[col].apply(lambda s: int(s))
for col in floats_a:
    df_a[col] = df_a[col].apply(lambda s: float(s))
for col in texts_a:
    df_a[col] = pd.factorize(df_a[col], sort=True)[0]
df_a[df_a.index < 3]

KeyError: "labels ['imei'] not contained in axis"

In [3]:
# обучающая выборка: набор пар номеров телефонов (msisdn)
# каждая из которых с большой долей вероятности принадлежит одной Персоне.
ints_b=["v_0", "v_1"]
df_b = pd.read_excel('../input/01_RU_fakty.xlsx', sep=";", header=None, sheet='Лист 1').rename(columns={0:"v_0",1:"v_1"})
for col in ints_b:
    df_b[col] = df_b[col].apply(lambda s: int(s))
df_b[df_b.index < 3]

,v_0,v_1
0,158530067653,158528861653
1,158510912201,158528852857
2,158530063945,158528861599


In [4]:
# Первые 8 цифр imei называются TAC и определяют устройство, которым пользуется абонент.
# Сокращенный справочник TAC (файл 03_устройства) состоит из полей.
cols_c=["tac","vendor","platform","type"]
ints_c=["tac"]
texts_c=["vendor","platform","type"]
df_c = pd.read_csv('../input/03_RU_ustroystva.csv',sep=',\s*',skiprows=1,names=cols_c)
for col in cols_c:
    df_c[col] = df_c[col].apply(lambda s: s.replace("\"",""))
for col in ints_c:
    df_c[col] = df_c[col].apply(lambda s: int(s))
for col in texts_c:
    df_c[col] = pd.factorize(df_c[col], sort=True)[0]
df_c[df_c.index < 3]

C:\SOFFFT\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,tac,vendor,platform,type
0,100429,1663,25,3
1,100430,1663,25,3
2,100431,1663,25,3


In [5]:
# Каждая регистрации на базовой станции имеет свой тип.
# Справочник типов регистраций с расшифровкой в файле 04_event_type. 
cols_d=["number","label","description"]
ints_d=["number"]
texts_d=["label","description"]
df_d = pd.read_excel('../input/04_event_type.xlsx', sep=";", skiprows=1, names=cols_d, sheet='Лист 1')
for col in ints_d:
    df_d[col] = df_d[col].apply(lambda s: int(s))
for col in texts_d:
    df_d[col] = pd.factorize(df_d[col], sort=True)[0]
df_d[df_d.index < 3]

,number,label,description
0,1,5,0
1,2,8,7
2,3,0,2


In [6]:
df_train = df_b
df_train = df_train.merge(df_a, left_on="v_0", right_on="msisdn", how="left")
df_train[df_train.index < 10]

,v_0,v_1,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle,tac1,tac2,tac3,tac4,tac5,tac6,tac7,tac8
0,158530067653,158528861653,7782,48428,158530067653,1.303401e+13,0,1369281266860,37.4625,55.7214,50.0,2,170.0,230.0,1,13,130,1303,13034,130340,1303400,13034008
1,158530067653,158528861653,7752,30479,158530067653,1.303401e+13,0,1369281821169,37.5049,55.7309,1246.0,2,290.0,350.0,1,13,130,1303,13034,130340,1303400,13034008
2,158530067653,158528861653,7742,58364,158530067653,1.303401e+13,0,1369281894211,37.4842,55.7393,50.0,2,86.5,133.5,1,13,130,1303,13034,130340,1303400,13034008
3,158530067653,158528861653,7742,26211,158530067653,1.303401e+13,1,1369282509891,37.5392,55.7668,628.0,2,290.0,350.0,1,13,130,1303,13034,130340,1303400,13034008
4,158530067653,158528861653,7799,20136,158530067653,1.303401e+13,0,1369283494251,37.5876,55.7912,1108.0,2,330.0,30.0,1,13,130,1303,13034,130340,1303400,13034008
5,158530067653,158528861653,7799,9705,158530067653,1.303401e+13,1,1369294037018,37.5788,55.7964,253.0,0,0.0,360.0,1,13,130,1303,13034,130340,1303400,13034008
6,158530067653,158528861653,7799,9705,158530067653,1.303401e+13,10,1369297463477,37.5788,55.7964,253.0,0,0.0,360.0,1,13,130,1303,13034,130340,1303400,13034008
7,158530067653,158528861653,7799,7173,158530067653,1.303401e+13,2,1369303333463,37.5730,55.8007,449.0,2,85.0,145.0,1,13,130,1303,13034,130340,1303400,13034008
8,158530067653,158528861653,7799,9705,158530067653,1.303401e+13,8,1369310811017,37.5788,55.7964,253.0,0,0.0,360.0,1,13,130,1303,13034,130340,1303400,13034008
9,158530067653,158528861653,7799,21582,158530067653,1.303401e+13,10,1369317711384,37.5777,55.7975,109.0,2,90.0,150.0,1,13,130,1303,13034,130340,1303400,13034008
